In [ ]:
import os
import sys
import warnings
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import mutual_info_classif
# from sklearn.ensemble import ExtraTreesClassifier
import xarray as xr
# import matplotlib.pyplot as plt
# import numpy as np
import pandas as pd
# import seaborn as sn
# import statsmodels.api as sm
# from ecgdetectors import Detectors
# from petastorm import make_reader
# from sklearn.metrics import auc,roc_curve,precision_recall_curve,roc_auc_score,RocCurveDisplay
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.model_selection import (RepeatedStratifiedKFold, cross_val_score,
#                                      train_test_split)
warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.append(os.path.join(os.getcwd(), ".."))
import shared_utils.utils_data as utils_data
from shared_utils import Logistic_reg_model
from metrics.main import save_metrics_to_xarray

path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"

In [ ]:
save_path = "/workspaces/ecg_evaluation/results"
name_method = ["Corr_interlead","Corr_intralead","wPMF","SNRECG","HR","Kurtosis","Flatline","TSD"]

if not os.path.exists(save_path):
    os.makedirs(save_path)
os.path.join(save_path,"")
if not "quality_metrics.nc" in os.listdir(save_path):
    print("Computing metrics")
    if not "ecg_data.nc" in os.listdir(save_path):
        ds_data = utils_data.format_data_to_xarray(path_petastorm, save_path)
    else:
        ds_data = xr.load_dataset(os.path.join(save_path,"ecg_data.nc"))

    ds_metrics = save_metrics_to_xarray(ds_data, name_method, save_path, verbose = True)
else:
    ds_metrics = xr.load_dataset(os.path.join(save_path,"quality_metrics.nc"))

In [ ]:
ds_filtered = ds_metrics.where(ds_metrics.data_quality != "unlabeled").dropna(dim = "id")

np_metrics = ds_filtered.quality_metrics.values
metrics_names = ds_filtered.metric_name.values.tolist()
np_label = ds_filtered.data_quality.values
##Opposite labelling : instead of labelling 1 as acceptable, we label 1 as unacceptable : 
opposite = True
reverseUNO_y = np_label.copy()
original_label = np_label.copy()
reverseUNO_y[np_label == "acceptable" ] = 0
reverseUNO_y[np_label == "unacceptable" ] = 1
reverseUNO_y = reverseUNO_y.astype(int)
original_label[np_label == "acceptable" ] = 1
original_label[np_label == "unacceptable" ] = 0
original_label = original_label.astype(int)

In [ ]:
HR_index = metrics_names.index("HR")
HR_metrics = np_metrics[:,:,HR_index].min(axis=1)
X = np_metrics.mean(axis = 1)
X[:,HR_index] = HR_metrics
df_X = pd.DataFrame(X, columns =metrics_names )
df_y_normal = pd.DataFrame(original_label, columns = ["y"])
df_y_reverse = pd.DataFrame(reverseUNO_y, columns = ["y"])

## First, Let's create all the folder necessary

In [ ]:
save_path = "/workspaces/maitrise/results"
column = ["Corr_interlead","HR","SNRECG","Corr_intralead"]
Logistic_reg_model.save_model_index(df_X,df_y_normal,save_path,cols = column,Model_name = "JMI_MI_selection_method")

## Folder Format : 

- Indexes : Only a Folder name Fold_CV with each Test fold results (call Test_Fold_k.csv with k the fold index)
- Models : 
    </br>   
    |-name_model.sav
    </br>
    |-Fold_CV (same orgnization than indexes)

## CSV format :

- Indexes : [Len_test_fold*3] with 1st column = Proba_label_0, 2nd column = Proba_label_1, 3r column = Ref_test_label
- Models : [Len_test_fold*4] with 1st column = Proba_label_0, 2nd column = Proba_label_1, 3rd column = predicted_model_test_label, 4th = ref_test_label

### Calculate Performance Model : 

If you want to calculate the performance of your model, use "Classification_report_model" with the following input :
- path toward the folder which contain all your models
- name of your model (format of a list in case you want to test mutlitple model at a time)
- a threshold argment (use T as a name variable). if not given, all the models performance will be calculated using max MCC threshold

In [ ]:
model_path = "/workspaces/maitrise/results/Models"
model_s = ["JMI_MI_selection_method"]
Logistic_reg_model.Classification_report_model(model_path,model_s)

## Calculate Performance Index : 

For the indexes, follow the same principle than before by using "Classification_report_index" (input in the same format as before):

In [ ]:
index_path = "/workspaces/maitrise/results/Indexes"
index_s = ["Corr_interlead"]
Logistic_reg_model.Classification_report_index(index_path,index_s)

## Get ROC PR curve of your model : 

This time, use "ROC_PR_CV_curve_model" with the following inputs:
- path toward the folder which contain all your models
- name of your model (format of a list in case you want to test mutlitple model at a time)
- pos_label = label from which you ant to calculate the performance (int :0 or 1, default =0)

In [ ]:
model_path = "/workspaces/maitrise/results/Models"
model_s = ["SQA_method"]
Logistic_reg_model.ROC_PR_CV_curve_model(model_path,model_s)

## Get ROC PR curve of your index(es) : 

Same input format as before instead you use "ROC_PR_CV_curve_indexes"

In [ ]:
index_path = "/workspaces/maitrise/results/Indexes"
index_s = ["wPMF"]
Logistic_reg_model.ROC_PR_CV_curve_index(index_path,index_s)

## Comparative ROC PR Curve for all indexes and models : 
for that, use "Global_comp_ROC_PR_mean_curve" with the following inputs :
- path toward the folder which contain all your models
- path toward the folder which contain all your indexes
- name of your model (format of a list in case you want to test mutlitple model at a time) (default empty list)
- name of your indexes (format of a list in case you want to test mutlitple indexes at a time) (default : empty list)
- pos_label = label from which you ant to calculate the performance (int :0 or 1, default =0)

In [ ]:
Logistic_reg_model.Global_comp_ROC_PR_mean_curve(model_path,index_path)